# Estimating Choo & Siow

In [ ]:
import gurobipy as grb
import pandas as pd
import numpy as np
import os
from scipy import optimize

In [ ]:
thePath = os.getcwd().split("veteran_students_mec_optim\\Pauline")[0]
n_singles = pd.read_csv(thePath + "\\data_mec_optim\\marriage-ChooSiow\\n_singles.txt", sep='\t', header = None)
marr = pd.read_csv(thePath + "\\data_mec_optim\\marriage-ChooSiow\\marr.txt", sep='\t', header = None)
navail = pd.read_csv(thePath + "\\data_mec_optim\\marriage-ChooSiow\\n_avail.txt", sep='\t', header = None)

In [ ]:
nbCateg = 25

In [ ]:
muhat_x0 = n_singles[0].iloc[0:nbCateg]
muhat_0y = n_singles[1].iloc[0:nbCateg]
muhat_xy = marr.iloc[0:nbCateg:,0:nbCateg]

In [ ]:
Nh = muhat_xy.values.sum()+muhat_x0.sum()+muhat_0y.sum()

In [ ]:
muhat_xy = muhat_xy / Nh 
muhat_x0 = muhat_x0 / Nh 
muhat_0y = muhat_0y / Nh

In [ ]:
n_x = muhat_xy.sum(axis = 1)+muhat_x0
m_y = muhat_xy.sum(axis = 0)+muhat_0y

In [ ]:
nbX = nbCateg
nbY = nbCateg

xs = np.repeat(range(1,nbX+1),nbY).reshape(nbX,nbY)
ys = np.repeat(range(1,nbY+1),nbX).reshape(nbX,nbY).T

phi1_xy = -((xs-ys)**2).flatten()
phimat = np.column_stack((phi1_xy,np.multiply(phi1_xy,(((xs+ys)/2)**2).flatten()),np.multiply(phi1_xy,(((xs+ys-2)/2)**2).flatten()),np.multiply(phi1_xy,((xs+ys-1)**2).flatten())))

In [ ]:
nbK = phimat.shape[1]
phimat_mean = phimat.mean(axis = 0)
phimat_stdev = phimat.std(axis = 0, ddof = 1)
phimat = ((phimat - phimat_mean).T/phimat_stdev[:,None]).T

In [ ]:
def ObjFunc(uvlambda):
    u_x = uvlambda[0:nbX]
    v_y = uvlambda[nbX:(nbX+nbY)]
    l = uvlambda[(nbX+nbY):(nbX+nbY+nbK)]
    
    Phi_xy = phimat.dot(l.reshape(nbK,1)).reshape(nbX, nbY)
    mu_xy = np.exp(((Phi_xy - u_x).T-v_y).T/2)
    mu_x0 = np.exp(-u_x)
    mu_0y = np.exp(-v_y)
    
    val = sum(np.multiply(n_x,u_x))+sum(np.multiply(m_y,v_y))-np.sum(np.multiply(muhat_xy.values,Phi_xy), axis = (0,1)) + 2*np.sum(mu_xy, axis =(0,1)) + sum(mu_x0) + sum(mu_0y)
    
    return val

In [ ]:
def grad_ObjFunc(uvlambda):
    u_x = uvlambda[0:nbX]
    v_y = uvlambda[nbX:(nbX+nbY)]
    l = uvlambda[(nbX+nbY):(nbX+nbY+nbK)]
    
    Phi_xy = phimat.dot(l.reshape(nbK,1)).reshape(nbX, nbY)
    mu_xy = np.exp(((Phi_xy - u_x).T-v_y).T/2)
    mu_x0 = np.exp(-u_x)
    mu_0y = np.exp(-v_y)
    
    grad_u = n_x - np.sum(mu_xy, axis = 0) - mu_x0
    grad_v = m_y - np.sum(mu_xy, axis = 1) - mu_0y
    grad_lambda = (mu_xy-muhat_xy.values).flatten()[:,None].T.dot(phimat)
    
    grad = np.concatenate((grad_u,grad_v,grad_lambda.flatten()))
    
    return grad

In [ ]:
outcome = optimize.minimize(ObjFunc,method = 'CG',jac = grad_ObjFunc, x0 = np.repeat(0,nbX+nbY+nbK))

In [ ]:
uvlambdahat =  outcome['x']
lambdahat = uvlambdahat[(nbX+nbY):(nbX+nbY+nbK)]
print(outcome)
print("")
print(ObjFunc(uvlambdahat))
print(lambdahat)